In [ ]:
!pip install pandas
!pip install pulp
import pandas as pd
import pulp

# データの読み込み

In [ ]:
data = "/content/drive/MyDrive/Colab Notebooks/配属実習アルゴリズム/python/sample_data2.xlsx"

In [ ]:
df_students.head()

In [ ]:
df_ng.info()

<class 'pandas.core.frame.DataFrame'>
Index: 88 entries, m2021001 to m2021088
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   名前      88 non-null     object
 1   NGリスト   2 non-null      object
dtypes: object(2)
memory usage: 4.1+ KB


In [ ]:
df_departments.head()

,配属先名,定員
0,配属先１,3
1,配属先２,6
2,配属先３,7
3,配属先４,3
4,配属先５,5


In [ ]:
department_caps

{'配属先１': 3,
 '配属先２': 6,
 '配属先３': 7,
 '配属先４': 3,
 '配属先５': 5,
 '配属先６': 6,
 '配属先７': 5,
 '配属先８': 3,
 '配属先９': 4,
 '配属先１０': 3,
 '配属先１１': 6,
 '配属先１２': 3,
 '配属先１３': 5,
 '配属先１４': 4,
 '配属先１５': 5,
 '配属先１６': 3,
 '配属先１７': 8,
 '配属先１８': 5,
 '配属先１９': 4,
 '配属先２０': 4}

# 変数の定義

In [ ]:
df_students = pd.read_excel(data, sheet_name='Sheet1')
df_ng = pd.read_excel(data, sheet_name='Sheet2', index_col=0)
df_departments = pd.read_excel(data, sheet_name='Sheet3')
# 定数の定義
num_students = len(df_students)
num_departments = len(df_departments)
department_caps = dict(zip(df_departments['配属先名'], df_departments['定員']))
#変数の定義
assign = pulp.LpVariable.dicts('assign', ((i, j) for i in range(num_students) for j in range(num_departments)), cat='Binary')

# 問題の定義

In [ ]:
# 問題の定義
problem = pulp.LpProblem('DepartmentalMatchingProblem', pulp.LpMinimize)
# 目的関数の定義
problem += pulp.lpSum([df_students.iloc[i, j+2] * assign[(i, j)] for i in range(num_students) for j in range(num_departments)])

In [ ]:
# 制約条件の定義
for i in range(num_students):
    problem += pulp.lpSum([assign[(i, j)] for j in range(num_departments)]) == 1
for j in range(num_departments):
    problem += pulp.lpSum([assign[(i, j)] for i in range(num_students)]) <= department_caps[df_departments.iloc[j, 0]]
for i in range(num_students):
    for j in range(num_departments):
        if df_ng.iloc[i, :].isin([df_students.iloc[k, 0] for k in range(num_students)]).any():
            ng_list = list(df_ng.iloc[i, :].dropna())
            for ng in ng_list:
                if df_students.iloc[i, 0] != ng:
                    problem += assign[(i, j)] + assign[(df_students[df_students['学籍番号'] == ng].index[0], j)] <= 1

# 最適化の実行
result=problem.solve(pulp.PULP_CBC_CMD( msg=1, threads=4, timeLimit=100))
print('result: {}'.format(pulp.LpStatus[result]))

result: Optimal


# dfの作製

In [ ]:
# prompt: assignの内容をdfにしたい

assign_df = pd.DataFrame(index=df_students['学籍番号'], columns=df_departments['配属先名'])
for i in range(num_students):
    for j in range(num_departments):
        if assign[(i, j)].value() == 1:
            assign_df.loc[df_students.iloc[i, 0], df_departments.iloc[j, 0]] = 1
        else:
            assign_df.loc[df_students.iloc[i, 0], df_departments.iloc[j, 0]] = 0

print(assign_df)

配属先名      内科 外科 小児科 産婦人科 眼科 耳鼻咽喉科 整形外科 泌尿器科 脳神経外科 循環器科 消化器科 精神科
学籍番号                                                           
m20210001  0  1   0    0  0     0    0    0     0    0    0   0
m20210002  0  0   0    0  0     0    0    1     0    0    0   0
m20210003  0  0   0    0  0     0    1    0     0    0    0   0
m20210004  0  0   0    0  0     1    0    0     0    0    0   0
m20210005  0  0   0    0  0     0    0    0     0    0    1   0
m20210006  0  0   0    0  0     0    0    0     0    1    0   0
m20210007  0  0   0    0  0     0    0    1     0    0    0   0
m20210008  0  0   0    0  1     0    0    0     0    0    0   0
m20210009  0  0   0    0  0     0    0    0     0    0    0   1
m20210010  0  0   0    0  0     1    0    0     0    0    0   0
m20210011  0  1   0    0  0     0    0    0     0    0    0   0
m20210012  0  0   1    0  0     0    0    0     0    0    0   0
m20210013  0  0   0    0  1     0    0    0     0    0    0   0
m20210014  1  0   0    0  0     0    0  

# いろいろな処理に関して

In [ ]:
# 結果の表示
for i in range(num_students):
    for j in range(num_departments):
        if assign[(i, j)].value() == 1:
            print(f"{df_students.iloc[i, 0]}: {df_students.iloc[i, 1]} -> {df_departments.iloc[j, 0]}")

# 生徒ごとの割り当て状況を辞書に格納する
assignments = {}
for i in range(num_students):
    for j in range(num_departments):
        if assign[(i, j)].value() == 1:
            if df_departments.iloc[j, 0] not in assignments:
                assignments[df_departments.iloc[j, 0]] = []
            assignments[df_departments.iloc[j, 0]].append(df_students.iloc[i, 1])

# 配属先ごとの名簿を表示する
for department in assignments:
    print(f"{department}:")
    for student in assignments[department]:
        print(f"  {student}")

# 生徒ごとの希望順位をリスト化する
student_prefs = []
for i in range(num_students):
    prefs = []
    for j in range(num_departments):
        if assign[(i, j)].value() == 1:
            prefs.append(j)
    student_prefs.append(prefs)

# 結果の表示
for i in range(num_students):
    prefs_str = ', '.join([f"第{n+1}希望に決定: {df_departments.iloc[p, 0]}" for n, p in enumerate(student_prefs[i])])
    print(f"{df_students.iloc[i, 0]}: {df_students.iloc[i, 1]} ({prefs_str})")


# クラス化

In [ ]:
class DepartmentalMatchingProblem:
    def __init__(self, data_file):
        self.data_file = data_file
        self.df_students, self.df_ng, self.df_departments, self.department_caps = self.load_data()

    def load_data(self):
        df_students = pd.read_excel(self.data_file, sheet_name='Sheet1')
        df_ng = pd.read_excel(self.data_file, sheet_name='Sheet2', index_col=0)
        df_departments = pd.read_excel(self.data_file, sheet_name='Sheet3')
        department_caps = dict(zip(df_departments['配属先名'], df_departments['定員']))
        return df_students, df_ng, df_departments, department_caps

    def solve(self):
        num_students = len(self.df_students)
        num_departments = len(self.df_departments)
        assign = pulp.LpVariable.dicts('assign', ((i, j) for i in range(num_students) for j in range(num_departments)), cat='Binary')
        problem = pulp.LpProblem('DepartmentalMatchingProblem', pulp.LpMinimize)
        problem += pulp.lpSum([self.df_students.iloc[i, j+2] * assign[(i, j)] for i in range(num_students) for j in range(num_departments)])
        for i in range(num_students):
            problem += pulp.lpSum([assign[(i, j)] for j in range(num_departments)]) == 1
        for j in range(num_departments):
            problem += pulp.lpSum([assign[(i, j)] for i in range(num_students)]) <= self.department_caps[self.df_departments.iloc[j, 0]]
        for i in range(num_students):
            for j in range(num_departments):
                if self.df_ng.iloc[i, :].isin([self.df_students.iloc[k, 0] for k in range(num_students)]).any():
                    ng_list = list(self.df_ng.iloc[i, :].dropna())
                    for ng in ng_list:
                        if self.df_students.iloc[i, 0] != ng:
                            problem += assign[(i, j)] + assign[(self.df_students[self.df_students['学籍番号'] == ng].index[0], j)] <= 1
        result = problem.solve(pulp.PULP_CBC_CMD(msg=1, threads=4, timeLimit=100))
        return result, assign

    def create_assignment_df(self, assign):
        num_students = len(self.df_students)
        num_departments = len(self.df_departments)
        assign_df = pd.DataFrame(index=self.df_students['学籍番号'], columns=self.df_departments['配属先名'])
        for i in range(num_students):
            for j in range(num_departments):
                if assign[(i, j)].value() == 1:
                    assign_df.loc[self.df_students.iloc[i, 0], self.df_departments.iloc[j, 0]] = 1
                else:
                    assign_df.loc[self.df_students.iloc[i, 0], self.df_departments.iloc[j, 0]] = 0
        return assign_df

    def print_results(self, assign):
        num_students = len(self.df_students)
        num_departments = len(self.df_departments)
        for i in range(num_students):
            for j in range(num_departments):
                if assign[(i, j)].value() == 1:
                    print(f"{self.df_students.iloc[i, 0]}: {self.df_students.iloc[i, 1]} -> {self.df_departments.iloc[j, 0]}")
        assignments = {}
        for i in range(num_students):
            for j in range(num_departments):
                if assign[(i, j)].value() == 1:
                    if self.df_departments.iloc[j, 0] not in assignments:
                        assignments[self.df_departments.iloc[j, 0]] = []
                    assignments[self.df_departments.iloc[j, 0]].append(self.df_students.iloc[i, 1])
        for department in assignments:
            print(f"{department}:")
            for student in assignments[department]:
                print(f"  {student}")
        student_prefs = []
        for i in range(num_students):
            prefs = []
            for j in range(num_departments):
                if assign[(i, j)].value() == 1:
                    prefs.append(j)
            student_prefs.append(prefs)
        for i in range(num_students):
            prefs_str = ', '.join([f"第{n+1}希望に決定: {self.df_departments.iloc[p, 0]}" for n, p in enumerate(student_prefs[i])])
            print(f"{self.df_students.iloc[i, 0]}: {self.df_students.iloc[i, 1]} ({prefs_str})")

    def to_dataframe(self, assign):
        num_students = len(self.df_students)
        num_departments = len(self.df_departments)
        assign_df = pd.DataFrame(index=self.df_students['学籍番号'], columns=self.df_departments['配属先名'])
        for i in range(num_students):
            for j in range(num_departments):
                if assign[(i, j)].value() == 1:
                    assign_df.loc[self.df_students.iloc[i, 0], self.df_departments.iloc[j, 0]] = 1
                else:
                    assign_df.loc[self.df_students.iloc[i, 0], self.df_departments.iloc[j, 0]] = 0
        return assign_df

In [ ]:
# Usage:
problem = DepartmentalMatchingProblem('/content/drive/MyDrive/Colab Notebooks/配属実習アルゴリズム/python/sample_data2.xlsx')
result, assign = problem.solve()
assign_df = problem.to_dataframe(assign)
problem.print_results(assign)